In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.io import loadmat
from scipy import signal
from pathlib import Path
import h5py
import utils
from mpl_markers import axis_marker
import sys
import utils
import mat73

dir_ = Path.cwd()

%config InlineBackend.figure_formats = ['svg']
from IPython.display import HTML

# ECE 6961 Wireless Communications
## Project Part 2

Group Members: Grant Brown, Thomas Warren, Rick Lyon

Setup
---------

To generate this report and the associated plots:

1. Install Python 3.11.
2. Open a terminal window and navigate to the directory containing the project files. 
2. Create an virtual environment for the report/project. See [here](https://docs.python.org/3/library/venv.html) for more information.  
`> python -m venv .venv`
4. In the newly created environment, install the required packages from the requirements.txt file included with the project files.  
`(.venv) > pip install -r requirements.txt`
5. Open the report with Jupyter Notebooks.  
`(.venv) > jupyter notebook part3.ipynb`
6. The notebook should open in the default browser. 

7. To generate the .html version of the report,  
`(.venv) > jupyter nbconvert --to html part3.ipynb`

In [16]:
# parameters
K = 2048 # number of subcarriers
L = 200 # number of zero pad samples
Fc = 24e3 # carrier frequency
B = 8e3 # bandwidth
W = 24 # number of OFDM symbols
sps = 24 # samples per symbol
fs_rx = 256e3 # sampling rate of recieiver
fs_tx = 192e3
Ts_rx = 1 / fs_rx
Ts_tx = 1 / fs_tx
k_p = 512
k_d = 1420
k_n = 112

############################################
pilot_data = mat73.loadmat("data/OFDM_PILOT.mat")
pilot_data = pilot_data["OFDM_PILOT"]

#Create pilot identity matrix
pilot_tap_matrix = np.eye(pilot_data.shape[0], dtype=np.complex64)
for i in range(0,pilot_data.shape[0]):
    pilot_tap_matrix[i][i] = pilot_data[i]
V = utils.dftmtx(pilot_tap_matrix.shape[0])

benchmark_zw_data_1 = mat73.loadmat("data/benchmark_Zw_172648_1.mat")
benchmark_zw_data_1 = benchmark_zw_data_1["bb_rece_data_172648_1474"]

ofdm_map = mat73.loadmat("data/ofdm_map.mat")
ofdm_map = ofdm_map["ofdm_map"]
pilot_indicies = np.argwhere(ofdm_map==1.0)
pilot_zw = np.empty([k_p, 1], dtype=np.complex64)
for i, p_i in enumerate(pilot_indicies):
    pilot_zw[i] = benchmark_zw_data_1[p_i,1]


### Task 1a - Pilot-based least squares channel estimation

In [14]:
test = np.asmatrix(V).getH()*np.asmatrix(pilot_tap_matrix).getH()
h_ls = (1/k_p)*(np.asmatrix(V).getH()*np.asmatrix(pilot_tap_matrix).getH())*(np.asmatrix(benchmark_zw_data_1[200:,:]))
V_h = utils.dftmtx(h_ls.shape[0])
H_LS_W = np.matmul(V_h, h_ls)

test


### Task 1b - One-Tap Equalizer

In [24]:
data_indicies = np.argwhere(ofdm_map==2.0)
data_zw = np.empty([k_d, 1], dtype=np.complex64)
for i, p_i in enumerate(data_indicies):
    data_zw[i] = benchmark_zw_data_1[p_i,1]

null_indicies = np.argwhere(ofdm_map==0.0)
null_zw = np.empty([k_n, 1], dtype=np.complex64)
for i, p_i in enumerate(null_indicies):
    null_zw[i] = benchmark_zw_data_1[p_i,1]

two_sigma_squared = (1/k_n)*(np.square(np.linalg.norm(null_zw)))

x_1 = 1/np.sqrt(2) + 1j*1/np.sqrt(2)
x_2 = 1/np.sqrt(2) - 1j*1/np.sqrt(2)
x_3 = 1/np.sqrt(2) - 1j*1/np.sqrt(2)
x_4 = -1/np.sqrt(2) - 1j*1/np.sqrt(2)

A_n = np.empty([k_d, 1], dtype=np.float64)
B_n_b0 = np.empty([k_d, 1], dtype=np.float64)
B_n_b1 = np.empty([k_d, 1], dtype=np.float64)
A_d_b0 = np.empty([k_d, 1], dtype=np.float64)
A_d_b1 = np.empty([k_d, 1], dtype=np.float64)
B_d = np.empty([k_d, 1], dtype=np.float64)

for i in range(data_zw.shape[0]):
    A_n[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_1))
    B_n_b0[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_3))
    B_n_b1[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_2))
    A_d_b0[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_2))
    A_d_b1[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_3))
    B_d[i] = np.square(np.linalg.norm(data_zw[i] - H_LS_W[data_indicies[i]]*x_4))

for i in range(data_zw.shape[0]):
    b_0_n = np.max(A_n[i], B_n_b0[i]) + np.log(1+np.exp(-1*np.abs(B_n_b0[i]-A_n[i])))
    b_0_d = np.max(A_d_b0[i], B_d[i]) + np.log(1+np.exp(-1*np.abs(B_d[i]-A_d_b0[i])))
    b_1_n = np.max(A_n[i], B_n_b1[i]) + np.log(1+np.exp(-1*np.abs(B_n_b1[i]-A_n[i])))
    b_1_d = np.max(A_d_b1[i], B_d[i]) + np.log(1+np.exp(-1*np.abs(B_d[i]-A_d_b1[i])))


for i in range(data_zw.shape[0]):
    pass

Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/signals/lib/python3.11/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 624, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined
